In [28]:
import numpy as np

In [29]:
#1. ACI
def compute_aci(specs, time_steps, freq_bins, num_specs):
      
    import numpy as np
    #Function computes Acoustic Complexity Index
    #According to:
    #http://www.iinsteco.org/people/pubblications/almo/2011/2011_A_new%20methodology_to_infer_the_singing_activity.pdf
    
    #Adding eps
    eps = 1e-10
    specs = specs + eps  
    
    abs_diff = np.abs(specs[:, :, 1:] - specs[:, :, 0:time_steps-1])
    ACI = np.sum(np.sum(abs_diff, axis = 2)/np.sum(specs, axis = 2), axis = 1)
    
    return ACI

In [30]:
#2. Acoustic diversity
def compute_adi(specs, time_steps):

    import numpy as np
    #Function computes Acoustic Diversity Index
    #According to: Pekin (2012)
    #Each bin corresponds to one of the 70 frequency points
   
    #Adding eps
    eps = 1e-10
    specs = specs + eps
    
    row_sums = specs.sum(axis=2, keepdims=False)
    all_sum = row_sums.sum(axis=1, keepdims=True)
    row_norm = row_sums/all_sum                                  #normalization
    ADI_all = (row_norm * np.log(row_norm)).sum(axis=1)

    return ADI_all


In [31]:
#Helper function for compute_adi_even
def get_start_stop_indices(freq_bins, multiples=1000, num_bins=8):
    
    import numpy as np
    #find the frequency index numbers to group in bins of 1kHz width from 0-1 kHz, 1-2, till 7-8kHz
    stop_freq_idx = np.zeros((num_bins, 1))
    start_freq_idx = np.zeros((num_bins,1))

    for i in range(num_bins):
        stop_freq = multiples*(i+1)
        #to find the index to clip
        stop_freq_idx[i] = freq_bins.searchsorted(stop_freq, side='right') - 1
        if (i<num_bins-1):
            start_freq_idx[i+1] = stop_freq_idx[i]+1
        
    return start_freq_idx, stop_freq_idx

In [32]:
#3. Acoustic diversity with evenly spaced bins
def compute_adi_even(num_bins, specs, freq_vals, start_indices, stop_indices):

    import numpy as np
    #Function computes Acoustic Diversity Index
    #According to: Pekin (2012)
    
    #Adding eps
    eps = 1e-10
    specs = specs+eps
  
    num_specs = specs.shape[0]
    ADI_bins = np.zeros((num_specs, num_bins))
    
    ADI_all = 0
    
    for i in range(num_bins):      
    
        #for loop for all time steps in same freq bin - sum each row
        start = int(start_indices[i])
        stop = int(stop_indices[i])
        specs_strip = specs[:, start:stop, :]
        ADI_bins[:, i] = np.sum(np.sum(specs_strip, axis = 2), axis = 1)
        
    #Normalizing by summing along axis 1: bin number
    ADI_bins_norm = ADI_bins/np.sum(ADI_bins, axis = 1, keepdims=True)
    
    #Computing
    ADI_all = (ADI_bins_norm * np.log(ADI_bins_norm)).sum(axis=1)
    
    return ADI_all

In [33]:
#4. Spectral entropy 
def compute_sh(specs, time_steps, freq_bin):
    
    import numpy as np
    #This function is a modified version of  spectral entropy as described in Sueur (2008)
    #Adding eps
    eps = 1e-10
    specs = specs+eps
    
    #Convert each frame to absolute value
    time_sums = np.sum(np.abs(specs), axis = 1, keepdims=True)
    spec_norm = np.divide(np.abs(specs), time_sums)

    #Added the eps in the log term, using np.multiply
    mult_term = np.multiply(spec_norm, np.log2(spec_norm + eps))
    
    #print("First frame multiplied: ", mult_term[0, :, 1])
   
    sh_all = - np.sum(mult_term.sum(axis = 1), axis = 1)
    sh_all = sh_all/np.log2(freq_bin)          #normalizing
    
    return sh_all

In [34]:
#5. NDSI
def compute_ndsi(spec, start_indices, stop_indices, start_a_freq, stop_a_freq, start_b_freq, stop_b_freq):
    
    import numpy as np
    #Taken from the Kasten 2012 paper - page 6 has NDSI description
    
    #Adding eps
    eps = 1e-10
    spec = spec + eps
    
    #Anthrophony bin: 1-2 kHz
    start_a = int(start_indices[start_a_freq])
    stop_a = int(stop_indices[stop_a_freq-1])
    
    #Biophony bin: 2-8 kHz
    start_b = int(start_indices[start_b_freq])
    stop_b = int(stop_indices[stop_b_freq-1])
    
    #Taking absolute value - power spectral density was squared, so intensities are made positive here

    anth_sum = np.sum(np.sum(np.abs(spec[:, start_a:stop_a, :]), axis = 2), axis = 1)
    bio_sum = np.sum(np.sum(np.abs(spec[:, start_b:stop_b, :]), axis = 2), axis = 1)
    NDSI = (bio_sum-anth_sum)/(bio_sum+anth_sum)
    
    return NDSI